# Эксперименты с habr доменом

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

In [59]:
from pathlib import Path
from tqdm import tqdm
import pickle
from time import time
from tqdm import tqdm
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer

from src.models.ContentBasedRecommenders import TagsBasedRecommender

In [3]:
data_path = Path('../data')

In [4]:
df_posts = pd.read_parquet(data_path / 'habr_posts.parquet')

In [5]:
df_posts

,id,url,title,tags
0,h729916,https://habr.com/ru/articles/729916/,3D анимация Флекса или простого танца через ци...,"Разработка игр, Компьютерная анимация, Дизайн игр"
1,h729914,https://habr.com/ru/articles/729914/,Расширяя границы: компания «Тантор Лабс» запус...,PostgreSQL
2,h729912,https://habr.com/ru/articles/729912/,МТС в рамках допсоглашения урегулировала задол...,"IT-инфраструктура, Управление продажами, Финан..."
3,h729910,https://habr.com/ru/articles/729910/,Кто все вот эти на полках: краткое руководство...,"Блог компании Сбер, Гаджеты, Ноутбуки"
4,h729904,https://habr.com/ru/articles/729904/,Information: Microsoft работает над ИИ-чипами,"Машинное обучение, Производство и разработка э..."
...,...,...,...,...
3305,h714926,https://habr.com/ru/articles/714926/,"Про Vim "" Клиент БД",VIM
3306,h714924,https://habr.com/ru/articles/714924/,Специалисты Microsoft обвинили связанных с Ира...,"Информационная безопасность, IT-компании"
3307,h714922,https://habr.com/ru/articles/714922/,Команда Google Chromium разрабатывает браузер ...,"Разработка под iOS, Браузеры"
3308,h714920,https://habr.com/ru/articles/714920/,Общие шаблоны и нюансы использования React Query,"JavaScript, ReactJS"


In [55]:
# Посты дял теста модели - про машинное обучение, конечно
test_posts = ['h729904', 'h714918']
df_posts[df_posts.id.isin(test_posts)]

,id,url,title,tags
4,h729904,https://habr.com/ru/articles/729904/,Information: Microsoft работает над ИИ-чипами,"Машинное обучение, Производство и разработка э..."
3309,h714918,https://habr.com/ru/articles/714918/,Как использовать GPT-3 для повышения эффективн...,"Машинное обучение, Контекстная реклама, Поиско..."


In [60]:
model = TagsBasedRecommender()

In [61]:
model.fit(df_posts.rename({'id': 'item'}, axis=1))

In [62]:
recs = model.recommend(test_posts, 20, mode='deterministic')

In [63]:
df_posts[df_posts.id.isin(recs)]

,id,url,title,tags
276,h729236,https://habr.com/ru/articles/729236/,"Глава OpenAI сообщил, что компания разрабатыва...","Машинное обучение, Искусственный интеллект, IT..."
431,h728888,https://habr.com/ru/articles/728888/,Италия предоставила OpenAI список действий для...,"Машинное обучение, Искусственный интеллект, IT..."
483,h728762,https://habr.com/ru/articles/728762/,"Первая языковая модель, которую наконец можно ...","Машинное обучение, Искусственный интеллект"
649,h728384,https://habr.com/ru/articles/728384/,Microsoft намекнула на скорое появление плагин...,"Машинное обучение, Искусственный интеллект, IT..."
886,h727832,https://habr.com/ru/articles/727832/,Infinityformer: новый подход к обработке длинн...,"Машинное обучение, Искусственный интеллект"
1444,h726590,https://habr.com/ru/articles/726590/,Специалисты по этике AI раскритиковали призыв ...,"Машинное обучение, Искусственный интеллект"
2387,h724432,https://habr.com/ru/articles/724432/,"OpenAI добавила в ChatGPT поддержку плагинов, ...","Машинное обучение, Искусственный интеллект, IT..."
2482,h724216,https://habr.com/ru/articles/724216/,Mozilla создала стартап Mozilla.ai для разрабо...,"Машинное обучение, Искусственный интеллект, IT..."
2701,h723724,https://habr.com/ru/articles/723724/,"Стартапы начали использовать GPT-4, чтобы мень...","Машинное обучение, Искусственный интеллект, IT..."
2812,h723482,https://habr.com/ru/articles/723482/,"Основатель OpenAI обеспокоен тем, что обновлён...","Машинное обучение, Искусственный интеллект, IT..."


In [64]:
recs = model.recommend(test_posts, 20, mode='probabilistic')

In [65]:
df_posts[df_posts.id.isin(recs)]

,id,url,title,tags
296,h729194,https://habr.com/ru/articles/729194/,В «Яндекс Лавке» теперь доступно более 100 тыс...,"Управление e-commerce, IT-компании"
304,h729178,https://habr.com/ru/articles/729178/,"Дайджест научпоп-новостей за неделю, о которых...","Научно-популярное, Энергия и элементы питания,..."
316,h729148,https://habr.com/ru/articles/729148/,Как работать с нейросетями и создать свой Chat...,"Блог компании Нетология, Учебный процесс в IT,..."
568,h728570,https://habr.com/ru/articles/728570/,Греф заявил о риске создания закрытого клуба к...,"Машинное обучение, Облачные сервисы, Искусстве..."
867,h727878,https://habr.com/ru/articles/727878/,Давид и Голиаф: плюсы работы в маленьких старт...,"Карьера в IT-индустрии, Офисы IT-компаний, Ист..."
886,h727832,https://habr.com/ru/articles/727832/,Infinityformer: новый подход к обработке длинн...,"Машинное обучение, Искусственный интеллект"
2077,h725124,https://habr.com/ru/articles/725124/,Apple Pro Weekly News (20.03 – 26.03.23),"Гаджеты, Компьютерное железо, Смартфоны, IT-ко..."
2606,h723932,https://habr.com/ru/articles/723932/,Что Bing AI знает о ChatGPT,"Блог компании Click.ru, Интернет-маркетинг, Ко..."
2817,h723472,https://habr.com/ru/articles/723472/,Amazon грозит судебный иск из-за нарушения зак...,"Законодательство в IT, IT-компании"
2995,h723084,https://habr.com/ru/articles/723084/,СМИ: фишинговых сайтов в РФ за год стало втрое...,"Информационная безопасность, Законодательство ..."
